In [1]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import numpy as np
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer

In [2]:
lis = ['GATAATGATCGCCGGGCCGGTAACA', 'AAGCATCTGGCTACGCTTTCCTTCA', 'AAGCAACACCAGTGCCATGACCATG', 'GCAGGTGCTCGCCGTCATAAAGGAC', 'GATCTTCCACCAATGCCGTGTAATA', 'AGTTGGAAGCGAATTCACGACATTC', 'CCCAAAGGATTTTTCTCGGCTGCCC', 'AAGTTGCGTAACAGCGAGACGTAGG', 'AGCGATATGGCCGAGGCAAGGGCAG', 'GAGGGAAGATATTGGCTGTTACGCA', 'ACATGGGATCTCGTCGCCGACATGG', 'ATCAGACTATGCATGAATGTCGCAG', 'GAAAAACTATATTGCGCGCTCGGCA', 'ATGAAATCGTAAGTCTCGCCGCCGC', 'CTTTCCACATTCCAATCCGCCGCAC', 'GAAGATCGCAGGGAGGACGTCTTCA', 'ACCGACCTTGAGCATTTTTCCCTCA', 'AAACCGGCTATGGCCCTTCAGGCCT', 'ATGAACAGGATGCCGACGATCCAGA', 'ACGCGACTCATGCCGGTGAAAGCCT', 'ACAAGGCGGCGACTTATGTTTATTC', 'ACCCAGTTCGCCCCACATCGCATCC', 'AAGCCATAGCCGCCGGTTCTGGTTC', 'CAGCGCAAGGACCCGGTCTTCATCG', 'AAGTGGTTTGGTCTGCGCACACCGG', 'TCCAGTTGGAAGCGAATTCACGACA', 'GCATAATCATAATGATAGTCGAGGC', 'TCGTCGACGTCCGATCTGCGCGCAA', 'CCATCGCGGCACTTTGGCCAGTGTC', 'CGGAAAAGATGCGCGTTAAAAACAA', 'GTTTCTCTCTCTTTTGAAAACTGCA', 'GGCCGGGCCGGAGATTTCCGGCGCC', 'CGCGAACTGTCGGCACGTCGCGATC', 'CCCGGAGCATATCATAGCCCCAGCC', 'CTTTCAAGGTTTCGGAATCGGAAAA', 'CACGAAGCATTCTGACAGAGCCACC', 'AGCGGCGATAGGCGGGGAGTTCGGC', 'CGTTTTGTCTCAGCCTACGCCGCTA', 'ACTTCTTGGGCTTTTCCCGGTCGCC', 'CTCAATCAAGGGATGGAACCGTTCC', 'GGCGCAGCCAAAGGCAAAATCCAGA', 'CTGTCAGTGAACTCGCGTCAGCCGC','TATTAAAATATATTATTCATAAATA', 'CTTATCGGCAAGACCGGCTCATGGC', 'TGACATTGACGGCGACGTCTCCGCA', 'GCTGACCAAGCGCATGGCGGAAGAC', 'ATTCCCGAATCACCAAACTGGTTCA', 'AACTGGTTTCGCTGACGGCATCTTG', 'ATCGGGCATCATGACATCGCTGATC', 'AAGCAGTCGGGCGAGGATTTCGGCA', 'AAACGGCCCAATGTCAGCTTGGCTT', 'ATGAAGTGGCGATGGTTGCCCTTCG', 'ATGATTGCCTCACGCTGGTCGTCCG', 'AATATGTTACAAAGCGTTATCAAGG', 'CTGCGTGGATTGCGTGACGTGGTTC', 'AAAGATCAACCTGATCGACCGTGCG', 'GATAAACTCGGTTGAAAAGCCCGCA', 'ACCATGTTGACGTCGGCAACCTGCG', 'ATTTCGCTTTGGCTGAGCTTGGAGC', 'ATGAAGGCGCACCGCGCAGGATAGC', 'CGGTTACCCAGTCGCTTTCCCAGAC', 'GCGAATTGTCCGGTATTTCCTTGCC', 'AAAGACAACGACATCAAGTTCGTCG', 'GCGTGCAAAATTGGGCGGCAGATCC', 'CGCCCGATGCCAATTTTATTGCATC', 'ACCACGCCGGAAAAGCGCCGGCGAG', 'ACTTGCCGGCATCGAGCGGTTCGCC', 'ACGACATGTCCGAAAACATCATCCA', 'GGCGGCAACCAGCAGAAGATCATCA', 'GGGATTGATGAGATAGCGCCCGGCA', 'ATAGACGCGCTTGGCCGCGAGATAA', 'AATCGTCGAGGATGGTCGACAGATA', 'ATTCTGGCCGAAGCCGCAATCGAGG', 'CGACATGCACGCCCTTGCCTTCAAG', 'ACATGGACAGGATGACCTTCGTGTC', 'AACCTTCTCGCGCATCCTCGACTGG', 'GACACATTGTGCCGAATGCCCGCCC', 'CGCACTGGTCAACGACCGTGGCGAG', 'CGTTCAGTTATTCAAATGGATGACA', 'CCAGCAAAAGGGCCTCTCACACCGC', 'GCGTCGGCTATCAGGTGACGGATTA', 'GACGTTGACGTGTCTGGCTCAAGAC', 'GCAGGCTCGTCGCTTTCCTTTGCCC', 'GCAAATCCGTTACTCAACGTCTGAA',
       'AAAGCCAGTTCAACCGCGCAACCAA', 'CATGTCCCGGCACGCGTTCACTCCG', 'GCGCCATTCGCGAAACCTTCGAGGA', 'GCAATCGTCGCATGTTCTACATTGA', 'CATGGCAGCCGGGTGTTTGATTCGG', 'CGGAATTTGCGTAGTCGCTCTTGAG', 'CCAGTTCATGATCGAGGAACTGGCG', 'CGGTGACGGCACGGGCGAGCAATGA', 'ATATCGAGCACGACCGCGTCATAAG', 'CCCGGTGATTGATGTCGTAGGTCAA', 'ATAAAGATGTAATCAAACAAAAGGC', 'ATGGGCGTCGGCGGGAACCGACGTG', 'AGCGGCAAATTCACATCTTATCAGG', 'CCCGGCTTGCTTGAAGCCATCATGA', 'GCTTCGGCAATGCCAATAAAGTCAC', 'CATCGGCATGAGCGGCGTTTTCCCG']

In [3]:
df = pd.read_csv("no_species.csv", nrows = 1000, usecols = lis)
df = df.astype('int8')

In [4]:
df_species = pd.read_parquet('final_df_species.parquet')
df_species = df_species.T
df_species.rename(columns={0: 'species'}, inplace=True)

In [5]:
data = df.join(df_species)
data

,GATAATGATCGCCGGGCCGGTAACA,AAGCATCTGGCTACGCTTTCCTTCA,AAGCAACACCAGTGCCATGACCATG,GCAGGTGCTCGCCGTCATAAAGGAC,GATCTTCCACCAATGCCGTGTAATA,AGTTGGAAGCGAATTCACGACATTC,CCCAAAGGATTTTTCTCGGCTGCCC,AAGTTGCGTAACAGCGAGACGTAGG,AGCGATATGGCCGAGGCAAGGGCAG,GAGGGAAGATATTGGCTGTTACGCA,...,CGGTGACGGCACGGGCGAGCAATGA,ATATCGAGCACGACCGCGTCATAAG,CCCGGTGATTGATGTCGTAGGTCAA,ATAAAGATGTAATCAAACAAAAGGC,ATGGGCGTCGGCGGGAACCGACGTG,AGCGGCAAATTCACATCTTATCAGG,CCCGGCTTGCTTGAAGCCATCATGA,GCTTCGGCAATGCCAATAAAGTCAC,CATCGGCATGAGCGGCGTTTTCCCG,species
GCF_900446045.1_58932_D01.genomic.fna.gz,0,1,0,0,0,0,0,0,1,0,...,0,0,0,1,1,1,0,0,0,Brucella abortus
GCF_005938105.1_ASM593810v1.genomic.fna.gz,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Brucella haematophila
GCF_000413955.1_Bruc_abor_biovar_1_B10-0091_V1.genomic.fna.gz,0,1,0,0,0,1,0,0,1,0,...,1,0,0,1,1,0,1,0,0,Brucella abortus
GCF_022024335.1_ASM2202433v1.genomic.fna.gz,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Brucella pseudogrignonensis
GCF_002191835.1_ASM219183v1.genomic.fna.gz,0,1,0,0,0,0,0,0,1,0,...,0,0,0,1,1,1,1,0,0,Brucella melitensis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GCF_003984745.1_ASM398474v1.genomic.fna.gz,0,1,0,0,0,0,0,0,1,0,...,1,0,0,1,1,0,1,0,0,Brucella abortus
GCF_947242555.1_B108.genomic.fna.gz,0,1,0,0,1,1,0,0,0,0,...,1,0,0,1,0,0,1,0,0,Brucella melitensis
GCF_000698345.1_ASM69834v1.genomic.fna.gz,0,1,0,0,1,0,0,0,1,0,...,1,0,0,1,1,1,1,0,0,Brucella suis
GCF_000332835.1_ASM33283v1.genomic.fna.gz,1,0,0,1,0,0,1,1,0,0,...,0,0,0,0,1,0,0,0,1,Brucella intermedia


In [6]:
train_df, test_df = train_test_split(data, test_size=0.10, random_state=123)
train_df

,GATAATGATCGCCGGGCCGGTAACA,AAGCATCTGGCTACGCTTTCCTTCA,AAGCAACACCAGTGCCATGACCATG,GCAGGTGCTCGCCGTCATAAAGGAC,GATCTTCCACCAATGCCGTGTAATA,AGTTGGAAGCGAATTCACGACATTC,CCCAAAGGATTTTTCTCGGCTGCCC,AAGTTGCGTAACAGCGAGACGTAGG,AGCGATATGGCCGAGGCAAGGGCAG,GAGGGAAGATATTGGCTGTTACGCA,...,CGGTGACGGCACGGGCGAGCAATGA,ATATCGAGCACGACCGCGTCATAAG,CCCGGTGATTGATGTCGTAGGTCAA,ATAAAGATGTAATCAAACAAAAGGC,ATGGGCGTCGGCGGGAACCGACGTG,AGCGGCAAATTCACATCTTATCAGG,CCCGGCTTGCTTGAAGCCATCATGA,GCTTCGGCAATGCCAATAAAGTCAC,CATCGGCATGAGCGGCGTTTTCCCG,species
GCF_002263015.1_ASM226301v1.genomic.fna.gz,0,0,0,0,1,0,0,0,1,0,...,1,0,0,1,1,0,1,0,0,Brucella melitensis
GCF_902825335.1_OCH-ISR1.genomic.fna.gz,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,Brucella intermedia
GCF_000366425.1_Bruc_abor_88_19_V1.genomic.fna.gz,0,0,0,0,1,1,0,0,1,0,...,0,0,0,1,1,1,1,0,0,Brucella abortus
GCF_000705115.1_Brucella_melitensis_C-555.genomic.fna.gz,0,1,0,0,0,1,0,0,1,0,...,0,0,0,1,0,1,0,0,0,Brucella melitensis
GCF_000160255.1_ASM16025v1.genomic.fna.gz,0,1,0,0,0,0,0,0,1,0,...,1,0,0,1,1,1,1,0,0,Brucella abortus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GCF_006507075.1_ASM650707v1.genomic.fna.gz,0,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,1,1,0,0,Brucella melitensis
GCF_000366525.1_Bruc_abor_CNGB_1432_V1.genomic.fna.gz,0,1,0,0,1,1,0,0,1,0,...,1,0,0,1,0,0,0,0,0,Brucella abortus
GCF_001856265.1_ASM185626v1.genomic.fna.gz,0,1,0,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,0,Brucella abortus
GCF_023651155.1_ASM2365115v1.genomic.fna.gz,0,1,0,0,1,1,0,0,1,0,...,1,0,0,1,0,0,0,0,0,Brucella abortus


In [7]:
test_df

,GATAATGATCGCCGGGCCGGTAACA,AAGCATCTGGCTACGCTTTCCTTCA,AAGCAACACCAGTGCCATGACCATG,GCAGGTGCTCGCCGTCATAAAGGAC,GATCTTCCACCAATGCCGTGTAATA,AGTTGGAAGCGAATTCACGACATTC,CCCAAAGGATTTTTCTCGGCTGCCC,AAGTTGCGTAACAGCGAGACGTAGG,AGCGATATGGCCGAGGCAAGGGCAG,GAGGGAAGATATTGGCTGTTACGCA,...,CGGTGACGGCACGGGCGAGCAATGA,ATATCGAGCACGACCGCGTCATAAG,CCCGGTGATTGATGTCGTAGGTCAA,ATAAAGATGTAATCAAACAAAAGGC,ATGGGCGTCGGCGGGAACCGACGTG,AGCGGCAAATTCACATCTTATCAGG,CCCGGCTTGCTTGAAGCCATCATGA,GCTTCGGCAATGCCAATAAAGTCAC,CATCGGCATGAGCGGCGTTTTCCCG,species
GCF_000370385.1_Bruc_abor_F10_05-11_V1.genomic.fna.gz,0,1,0,0,1,1,0,0,0,0,...,0,0,0,1,1,1,0,0,0,Brucella abortus
GCF_016126205.1_ASM1612620v1.genomic.fna.gz,0,0,0,0,1,1,0,0,1,0,...,0,0,0,1,0,0,1,0,0,Brucella abortus
GCF_947242625.1_B128.genomic.fna.gz,0,1,0,0,1,1,0,0,1,0,...,1,0,0,1,1,1,0,0,0,Brucella melitensis
GCF_000366285.1_Bruc_abor_64_122_V1.genomic.fna.gz,0,1,0,0,1,1,0,0,1,0,...,0,0,0,1,0,1,1,0,0,Brucella abortus
GCF_947242375.1_B9.genomic.fna.gz,0,1,0,0,1,1,0,0,0,0,...,0,0,0,1,1,0,1,0,0,Brucella melitensis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GCF_016483965.1_ASM1648396v1.genomic.fna.gz,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,1,1,1,0,0,Brucella abortus
GCF_900236365.1_Bru_genome40.genomic.fna.gz,0,1,0,0,1,1,0,0,1,0,...,1,0,0,1,0,1,1,0,0,Brucella melitensis
GCF_900417245.1_00-3099-5734.genomic.fna.gz,0,0,0,0,1,1,0,0,1,0,...,1,0,0,0,1,1,1,0,0,Brucella melitensis
GCF_023651395.1_ASM2365139v1.genomic.fna.gz,0,1,0,0,1,0,0,0,0,0,...,1,0,0,0,1,1,1,0,0,Brucella abortus


In [8]:
x_train = train_df.drop(columns=["species"])
y_train = train_df["species"]
x_test = test_df.drop(columns=["species"])
y_test = test_df["species"]

In [9]:
def mean_std_cross_val_scores(model, x_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, x_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [10]:
pipe_xgb = make_pipeline(XGBClassifier(learning_rate=0.1, max_depth=10,eval_metric = 'mlogloss'))
pipe_rf = make_pipeline(RandomForestClassifier(random_state=123, n_jobs=-1))

In [11]:
results = {}
results["Random forests"] = mean_std_cross_val_scores(
    pipe_rf, x_train, y_train, return_train_score=True, scoring='accuracy'
)
pd.DataFrame(results).T

/hobbitholes/waymoot/hchen/micromamba/envs/ach.0.8/lib/python3.7/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


,fit_time,score_time,test_score,train_score
Random forests,0.613 (+/- 0.751),0.105 (+/- 0.002),0.832 (+/- 0.025),0.993 (+/- 0.002)


In [12]:
results["XGboost"] = mean_std_cross_val_scores(
    pipe_xgb, x_train, y_train, return_train_score=True, scoring='accuracy'
)
pd.DataFrame(results).T

/hobbitholes/waymoot/hchen/micromamba/envs/ach.0.8/lib/python3.7/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/hobbitholes/waymoot/hchen/micromamba/envs/ach.0.8/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/hobbitholes/waymoot/hchen/micromamba/envs/ach.0.8/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the followi

,fit_time,score_time,test_score,train_score
Random forests,0.613 (+/- 0.751),0.105 (+/- 0.002),0.832 (+/- 0.025),0.993 (+/- 0.002)
XGboost,2.802 (+/- 0.080),0.008 (+/- 0.000),0.842 (+/- 0.031),0.986 (+/- 0.003)
